# **App scraper**

## **App Store - Apple**

In [1]:
import requests
import csv
import time
import logging

In [2]:
# Configure basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

**Configuration**

In [3]:
# Keywords to search for related to asthma
KEYWORDS = [
    'asthma',
    'asthma tracker',
    'inhaler tracker',
    'peak flow',
    'asthma management',
    'asthma log',
    'allergy induced asthma',
    'asthma symptoms',
    'asthma monitoring app',
    'asthma action plan'
]

In [4]:
# The base URL for the iTunes Search API
API_BASE_URL = 'https://itunes.apple.com/search'

# Output CSV file name
CSV_FILE = 'data/app_store_asthma_apps.csv'

# Number of top results to fetch for each keyword
RESULT_LIMIT = 10

**Main Scraper Function**

In [5]:
def scrape_app_store():
    """
    Scrapes the Apple App Store using the iTunes Search API for a list of
    keywords and saves the top 5 results for each to a CSV file.
    """
    logging.info(f"Starting App Store scrape. Output will be saved to {CSV_FILE}")

    csv_headers = [
        'Keyword Searched',
        'App Name',
        'App ID',
        'Seller',
        'Price',
        'Average Rating',
        'Rating Count',
        'Genre',
        'App URL'
    ]

    try:
        with open(CSV_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(csv_headers)

            for keyword in KEYWORDS:
                logging.info(f"Searching for keyword: '{keyword}'...")

                params = {
                    'term': keyword,
                    'country': 'US',
                    'media': 'software',
                    'limit': RESULT_LIMIT
                }

                try:
                    response = requests.get(API_BASE_URL, params=params, timeout=10)
                    
                    response.raise_for_status() 

                    data = response.json()
                    results = data.get('results', [])

                    if not results:
                        logging.warning(f"No results found for keyword: '{keyword}'")
                        continue

                    for app in results:
                        writer.writerow([
                            keyword,
                            app.get('trackName'),
                            app.get('trackId'),
                            app.get('artistName'),
                            app.get('formattedPrice', app.get('price', 'N/A')),
                            app.get('averageUserRating', 'N/A'),
                            app.get('userRatingCount', 'N/A'),
                            app.get('primaryGenreName', 'N/A'),
                            app.get('trackViewUrl')
                        ])

                    logging.info(f"Successfully saved {len(results)} apps for '{keyword}'")

                except requests.exceptions.RequestException as e:
                    logging.error(f"HTTP Request failed for keyword '{keyword}': {e}")
                except Exception as e:
                    logging.error(f"An error occurred processing keyword '{keyword}': {e}")

                time.sleep(2)

    except IOError as e:
        logging.critical(f"Failed to open or write to CSV file {CSV_FILE}: {e}")
        return

    logging.info(f"--- Scraping complete. Data saved to {CSV_FILE} ---")

In [6]:
scrape_app_store()

2025-11-05 19:59:08,740 - INFO - Starting App Store scrape. Output will be saved to data/app_store_asthma_apps.csv
2025-11-05 19:59:08,743 - INFO - Searching for keyword: 'asthma'...
2025-11-05 19:59:09,013 - INFO - Successfully saved 10 apps for 'asthma'
2025-11-05 19:59:11,015 - INFO - Searching for keyword: 'asthma tracker'...
2025-11-05 19:59:11,196 - INFO - Successfully saved 10 apps for 'asthma tracker'
2025-11-05 19:59:13,197 - INFO - Searching for keyword: 'inhaler tracker'...
2025-11-05 19:59:13,440 - INFO - Successfully saved 10 apps for 'inhaler tracker'
2025-11-05 19:59:15,442 - INFO - Searching for keyword: 'peak flow'...
2025-11-05 19:59:15,621 - INFO - Successfully saved 10 apps for 'peak flow'
2025-11-05 19:59:17,623 - INFO - Searching for keyword: 'asthma management'...
2025-11-05 19:59:17,831 - INFO - Successfully saved 9 apps for 'asthma management'
2025-11-05 19:59:19,833 - INFO - Searching for keyword: 'asthma log'...
2025-11-05 19:59:20,073 - INFO - Successfully s

---

## **Google Play - Android**

In [ ]:
from google_play_scraper import search
import csv

In [ ]:
KEYWORDS = [
    "asthma",
    "asthma tracker",
    "breathing monitor",
    "inhaler reminder",
    "peak flow",
    "bronchial asthma",
]

In [ ]:
def fetch_apps(keyword: str, limit: int = 20):
    return search(keyword, lang="en", country="us", n_hits=limit)

In [ ]:
def extract_info(app, keyword):
    return {
        "keyword": keyword,
        "app_id": app.get("appId"),
        "app_title": app.get("title"),
        "developer": app.get("developer"),
        "rating": app.get("score"),
        "installs": app.get("installs"),
        "price": app.get("priceText"),
        "genre": app.get("genre"),
        "summary": app.get("summary"),
        "content_rating": app.get("contentRating"),
        "url": app.get("url"),
    }

In [ ]:
def main():
    results = []
    for kw in KEYWORDS:
        apps = fetch_apps(kw)
        results.extend(extract_info(a, kw) for a in apps)

    if not results:
        print("no results found")
        return

    with open("asthma_apps.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=results[0].keys())
        writer.writeheader()
        writer.writerows(results)

    print("DONE. Saved to asthma_apps.csv")

if __name__ == "__main__":
    main()
